In [1]:
import os
import pandas as pd
import re
import seaborn as sns

In [2]:
os.chdir("/fs/scratch/PAS0439/Ming/virome_ecology_core_prkaryotes/results/04_prophage_rumen_mags/")

In [3]:
host = pd.read_csv("rumen_mags_prophage_summary.csv")

unclassified = list(host.query('Species != Species').index)
for index, row in host.iterrows():
    if index in unclassified:
        host.loc[index, "Species"] = "Unclassfied " + str(host.loc[index, "Genera"])

In [4]:
with open("prophage_rumen_mags_category3.txt", 'r') as file:
    category3 = file.read().split("\n")

In [5]:
core_genus = set(pd.read_csv("../R_project/core_genus_taxa.csv")["Genera"])
core_species = set(pd.read_csv("../R_project/core_species_taxa.csv")["Species"])
archaea_phylum = ["Methanobacteriota", "Thermoplasmatota"]

In [6]:
n_host_species_bac = len(set(host.query('Genera in @core_genus').query('Phylum not in @archaea_phylum').Species))
n_host_genome_bac = len(set(host.query('Genera in @core_genus').query('Phylum not in @archaea_phylum').user_genome))
n_host_genome_bac_with_prophage = len(set(host.query('Genera in @core_genus').query('Phylum not in @archaea_phylum').query('prophage in @category3').user_genome))
n_host_species_bac_with_prophage = len(set(host.query('Genera in @core_genus').query('Phylum not in @archaea_phylum').query('prophage in @category3').Species))

In [7]:
host = host.query('Genera in @core_genus').query('Phylum not in @archaea_phylum')

In [8]:
n_host_genome_bac

3620

In [9]:
n_host_species_bac

1388

In [10]:
n_host_species_bac_with_prophage

628

In [39]:
n_host_genome_bac_with_prophage

1128

In [11]:
n_prophage_per_species = host.groupby("Species").prophage.count().reset_index()

In [12]:
with open("non-cryptic-prophage.txt", 'r') as file:
    active_prophage = file.read().split("\n")

In [13]:
def _(x):
    if x in active_prophage:
        return(True)
    else:
        return(False)

In [14]:
host.loc[host[list(host.prophage.apply(_))].index, "non_cryptic"] = "True"

In [15]:
bac_for_tree = set(host.query('prophage in @category3').query('Phylum not in @archaea_phylum').query('Genera in @core_genus').Species)

In [16]:
bac_genome_for_tree = []
for f in bac_for_tree:
    genome =  list(host.query('Species == @f')['user_genome'])[0] 
    bac_genome_for_tree.append(genome)

In [17]:
bac_genome_for_tree_df =  host.query('user_genome in @bac_genome_for_tree')[['Species', 'user_genome', "Phylum"]].drop_duplicates()

In [18]:
n_non_cryptic_per_species = host.groupby("Species").non_cryptic.count().reset_index()

In [19]:
annotate_pre = pd.merge(bac_genome_for_tree_df, n_non_cryptic_per_species, left_on = "Species", right_on = "Species")

In [20]:
tmp1 = host.query('Species in @bac_for_tree').groupby("Species").prophage.count().reset_index().rename(columns = {"prophage":"n_prophage"})

In [21]:
tmp2 = host.query('Species in @bac_for_tree').groupby("Species").user_genome.count().reset_index().rename(columns = {"user_genome":"n_user_genome"})

In [22]:
prophage_rate = pd.merge(tmp1, tmp2, left_on = "Species", right_on = "Species")

In [23]:
prophage_rate["prophage_rate"] = prophage_rate.n_prophage/prophage_rate.n_user_genome

In [24]:
prophage_rate_df = prophage_rate.sort_values("prophage_rate", ascending=False)

In [25]:
annotate_df = pd.merge(annotate_pre, prophage_rate_df, left_on = "Species", right_on = "Species" )

In [26]:
annotate_df["non_cryptic_rate"] = annotate_df.non_cryptic/annotate_df.n_prophage

In [27]:
annotate_df

,Species,user_genome,Phylum,non_cryptic,n_prophage,n_user_genome,prophage_rate,non_cryptic_rate
0,Bifidobacterium boum,Act.CBR.10825.genomic,Actinobacteriota,1,4,6,0.666667,0.25
1,Bifidobacterium ruminantium,Act.CBR.12664.genomic,Actinobacteriota,0,1,2,0.500000,0.00
2,Prevotella sp900315455,BAC.CFR.1051.genomic,Bacteroidota,0,3,7,0.428571,0.00
3,F23-D06 sp002392325,BAC.CFR.1052.genomic,Bacteroidota,0,2,7,0.285714,0.00
4,Prevotella sp902780635,BAC.CFR.1092.genomic,Bacteroidota,0,2,2,1.000000,0.00
...,...,...,...,...,...,...,...,...
623,Treponema_D sp017550685,shenzhen.Jerseybin.126,Spirochaetota,1,2,2,1.000000,0.50
624,Ruminococcus_E sp902773035,shenzhen.Limousinbin.1789,Firmicutes_A,0,1,1,1.000000,0.00
625,Prevotella sp902780025,shenzhen.Limousinbin.2246,Bacteroidota,1,2,2,1.000000,0.50
626,CAG-83 sp902782615,x220.59,Firmicutes_A,0,2,2,1.000000,0.00


In [28]:
len(set(annotate_df.Phylum))

9

In [29]:
print(sns.color_palette("tab10").as_hex())

['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']


In [30]:
palette = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'][0:9]

In [31]:
phylum_color = {}
for f in range(len(set(annotate_df.Phylum))):
    phylum_color[list(set(annotate_df.Phylum))[f]] = palette[f]
    

In [32]:
phylum_color

{'Actinobacteriota': '#1f77b4',
 'Proteobacteria': '#ff7f0e',
 'Firmicutes': '#2ca02c',
 'Firmicutes_A': '#d62728',
 'Firmicutes_C': '#9467bd',
 'Fibrobacterota': '#8c564b',
 'Bacteroidota': '#e377c2',
 'Spirochaetota': '#7f7f7f',
 'Desulfobacterota_I': '#bcbd22'}

In [33]:
for index, row in annotate_df.iterrows():
    phylum = row["Phylum"]
    Species = row["Species"]
    color = phylum_color[phylum]
    annotate_df.loc[index, "Phylum_color"] = color
    if Species in core_species:
        annotate_df.loc[index, "core_species"] = True
    else:
        annotate_df.loc[index, "core_species"] = False
        

In [34]:
annotate_df

,Species,user_genome,Phylum,non_cryptic,n_prophage,n_user_genome,prophage_rate,non_cryptic_rate,Phylum_color,core_species
0,Bifidobacterium boum,Act.CBR.10825.genomic,Actinobacteriota,1,4,6,0.666667,0.25,#1f77b4,False
1,Bifidobacterium ruminantium,Act.CBR.12664.genomic,Actinobacteriota,0,1,2,0.500000,0.00,#1f77b4,False
2,Prevotella sp900315455,BAC.CFR.1051.genomic,Bacteroidota,0,3,7,0.428571,0.00,#e377c2,True
3,F23-D06 sp002392325,BAC.CFR.1052.genomic,Bacteroidota,0,2,7,0.285714,0.00,#e377c2,False
4,Prevotella sp902780635,BAC.CFR.1092.genomic,Bacteroidota,0,2,2,1.000000,0.00,#e377c2,False
...,...,...,...,...,...,...,...,...,...,...
623,Treponema_D sp017550685,shenzhen.Jerseybin.126,Spirochaetota,1,2,2,1.000000,0.50,#7f7f7f,False
624,Ruminococcus_E sp902773035,shenzhen.Limousinbin.1789,Firmicutes_A,0,1,1,1.000000,0.00,#d62728,False
625,Prevotella sp902780025,shenzhen.Limousinbin.2246,Bacteroidota,1,2,2,1.000000,0.50,#e377c2,True
626,CAG-83 sp902782615,x220.59,Firmicutes_A,0,2,2,1.000000,0.00,#d62728,False


In [35]:
with open("itol_host_phylum_color_strip.txt", "w") as outfile:
    with open("itol_host_phylum_color_strip_template.txt", "r") as infile:
        outfile.write(infile.read())
        outfile.write('\n')
        for index, row in annotate_df.iterrows():
            user_genome = row['user_genome']
            phylum = row['Phylum_color']
            outfile.write(user_genome + ' ' + phylum + "\n")

In [36]:
with open("itol_prophage_per_genome.txt", "w") as outfile:
    with open("itol_prophage_gradient_template.txt", "r") as infile:
        outfile.write(infile.read())
        outfile.write('\n')
        for index, row in annotate_df.iterrows():
            user_genome = row['user_genome']
            prophage_rate = row['prophage_rate']
            outfile.write(user_genome + ' ' + str(prophage_rate) + "\n")

In [37]:
with open("itol_non_cryptic_per_prophage.txt", "w") as outfile:
    with open("itol_prophage_gradient_template.txt", "r") as infile:
        outfile.write(infile.read())
        outfile.write('\n')
        for index, row in annotate_df.iterrows():
            user_genome = row['user_genome']
            non_cryptic_rate = row['non_cryptic_rate']
            outfile.write(user_genome + ' ' + str(non_cryptic_rate) + "\n")

In [38]:
with open("itol_prophage_core_species.txt", "w") as outfile:
    with open("itol_prophage_core_species_template.txt", "r") as infile:
        outfile.write(infile.read())
        outfile.write('\n')
        for index, row in annotate_df.iterrows():
            user_genome = row['user_genome']
            core_species = row['core_species']
            if core_species:
                outfile.write(user_genome + ',' + str(1) + "\n")